# PixelSounds: Visual Audio Encoding


## Introducción: Codificación de Audio en Vídeo con PixelSounds



PixelSounds es un sistema que convierte una señal de audio en un vídeo compuesto por frames. Cada frame codifica un bloque temporal del audio en forma de imagen, permitiendo visualizar la información sonora a través de representaciones visuales. El proceso es completamente reversible: se puede reconstruir el audio original a partir del vídeo.

### Modos de codificación (`map_mode`)

Cada bloque de audio se transforma en una fila de píxeles mediante uno de los siguientes modos:

- **`ampl`**  
  Codifica la amplitud normalizada del audio en una escala de grises.

- **`fft`**  
  Codifica el espectro en frecuencia (magnitud y fase) usando la Transformada Rápida de Fourier (FFT). Requiere codificar dos componentes por muestra.

- **`fir`**  
  Separa el audio en tres bandas de frecuencia (baja, media y alta) usando filtros FIR, y codifica cada banda en uno de los canales de color (R, G, B).

### Modos de visualización (`color_mode`)

Los frames generados pueden representarse en dos formas:

- **`gris`**  
  Las filas codificadas se replican verticalmente como una imagen monocroma. Este modo se usa, por ejemplo, para visualizar la amplitud o una sola banda.

- **`color`**  
  Se colorea cada muestra según su valor en los tres canales RGB, permitiendo ver diferentes componentes (por ejemplo, magnitud y fase, o bandas FIR) en diferentes colores.

### Estructura del sistema

- **Codificador:**  
  Fragmenta el audio, aplica la codificación elegida y genera una secuencia de imágenes PNG. Luego empaqueta los frames en un vídeo MP4 usando FFmpeg.

- **Decodificador:**  
  Extrae los frames del vídeo, reconstruye los bloques de audio a partir de las imágenes, y aplica overlap-add para recuperar la señal completa.

Este sistema permite experimentar con distintas formas de representar el contenido sonoro de una señal de audio, así como aplicar técnicas de visualización y análisis sobre los vídeos generados.


In [ ]:
# === IMPORTS ===
import os
import shutil
import subprocess
import numpy as np
import cv2
import imageio.v2 as imageio
from scipy.signal import firwin, lfilter, get_window
from scipy.fft import fft, ifft
from scipy.io import wavfile as wav
from IPython.display import Audio, Video, display

from vozyaudio import lee_audio, sonido
from funciones_rick import PixelSoundsEncoder, PixelSoundsDecoder

In [ ]:
for d in ("fotogramas", "exports"):
    if os.path.exists(d):
        print(f"Eliminando carpeta existente: {d}/")
        shutil.rmtree(d)
    else:
        print(f"No existe: {d}/ — nada que borrar.")

## 1. Carga de audio

Cargamos un WAV de ejemplo y mostramos sus primeros segundos.

In [ ]:
audio_path = "audios\music.wav"

fs, orig = lee_audio(audio_path)
print(f"Frecuencia de muestreo: {fs} Hz")
Audio(orig, rate=fs)

## 2. Definición de parámetros

### Relación entre FPS y FS en PixelSounds

Una parte clave del sistema PixelSounds es la relación entre la frecuencia de muestreo del audio (`fs`) y los fotogramas por segundo (`fps`) del vídeo generado. Esta relación determina cómo se divide la señal de audio en bloques temporales y, por tanto, qué resolución temporal tendrá la visualización.

#### Conversión audio → vídeo

- El audio original tiene una frecuencia de muestreo `fs` (por ejemplo, 44100 Hz).
- El vídeo tendrá `fps` (por ejemplo, 60 fotogramas por segundo).
- Cada frame del vídeo representará un bloque de **N = fs / fps** muestras de audio.

> Por ejemplo, si `fs = 44100` Hz y `fps = 60`, entonces `N = 735`.  
> Esto significa que cada fotograma representa 735 muestras de audio, es decir, 735 / 44100 = 0.0166 segundos ≈ 16.6 ms de sonido.

#### Ventaneo y solapamiento

- Para evitar discontinuidades, se utiliza un solapamiento entre bloques con salto `hop = N // 2` (solapamiento del 50%).
- Cada bloque se multiplica por una ventana (por ejemplo, tipo `hann`) antes de ser codificado para minimizar artefactos.

#### Efecto visual y auditivo

- Un valor alto de `fps` produce más frames por segundo y, por tanto, bloques de audio más pequeños (mayor resolución temporal, más precisión visual).
- Un valor bajo de `fps` produce menos frames por segundo y bloques más grandes (más eficiencia, pero menor detalle en la visualización).

> Es crucial mantener `fs` y `fps` constantes durante todo el proceso para asegurar que la reconstrucción del audio a partir del vídeo sea coherente y sin errores.

In [ ]:
fps         = 60
window_type = "hann"  # otras opciones: "hamming", "blackman", "bartlett", "kaiser", "boxcar", "triang", "nuttall", "flattop", "parzen", "bohman"

## 3. Modo Amplitude (`ampl`)


### Codificación

- **Bloque y ventana**  
  Se extrae un fragmento de \( N \) muestras del audio y se aplica una ventana (por ejemplo, Hann) para evitar discontinuidades en los bordes.

- **Normalización a [0–1]**  
  $$
  \text{norm} = \frac{\text{block} - \min(\text{block})}{\max(\text{block}) - \min(\text{block}) + \varepsilon}
  $$  
  Esto asegura que la amplitud mínima del bloque mapee a 0 y la máxima a 1.

- **Cuantización a 8 bits**  
  $$
  \text{fila}_i = \lfloor \text{norm}_i \times 255 \rfloor \quad \text{con} \quad 0 \leq \text{fila}_i \leq 255
  $$

- **Construcción de la imagen**  
  - **Escala de grises**: se replica la misma fila de \( N \) píxeles en cada una de las \( N \) filas del PNG, resultando en una imagen uniforme.
  - **Modo color**:  
    $$
    R = \text{amplitud}, \quad G = 255 - \text{amplitud}, \quad B = 128
    $$

In [ ]:
# 3.1 Generar → Empaquetar → Decodificar para ampl (gris y color)
map_mode = "ampl"
for color in ("gris", "color"):
    # 3.1.1 Carpetas de salida
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    os.makedirs(frames_dir, exist_ok=True)
    os.makedirs(export_dir, exist_ok=True)

    # 3.1.2 CODIFICAR
    enc = PixelSoundsEncoder(
        audio_path=audio_path,
        frames_dir=frames_dir,
        export_dir=export_dir,
        fps=fps,
        color_mode=color,
        map_mode=map_mode,
        window_type=window_type
    )
    enc.generate_frames()

    # 3.1.3 EMPAQUETAR VÍDEO
    video_file = enc.encode_video()
    print(f"[Notebook] Vídeo generado en: {video_file}")
    display(Video(video_file, embed=True, width=480, height=360))


---

#### Decodificación


- **Lectura de píxeles**  
  Se carga el PNG y se extrae la fila 0:  
  - En gris → valor único por píxel  
  - En color → canal Rojo

- **Desnormalización a [0–1]**  
  $$
  \text{amp} = \frac{\text{pixel}}{255}
  $$

- **Recuperar bipolaridad [–1 … +1]**  
  $$
  \text{block}_i = \text{amp}_i \times 2 - 1
  $$

- **Overlap-add con ventana**  
  Cada bloque reconstruido se solapa a la mitad (hop = \( N/2 \)) y se suma usando la misma ventana, recomponiendo la señal continua.

- **Normalización final**  
  $$
  \text{audio}[t] \mathrel{{/}{=}} \sum \text{ventana}
  $$

El resultado es un archivo `.wav` cuya forma de onda sigue fielmente la envolvente original, con las únicas pérdidas debidas a la cuantización a 8 bits y al solapamiento de ventanas.

In [ ]:
map_mode = "ampl"
base_name = os.path.splitext(os.path.basename(audio_path))[0]

for color in ("gris", "color"):
    # 1) Directorios y rutas
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    # Nombre del vídeo que acabamos de generar
    video_file = os.path.join(export_dir, f"{base_name}_{map_mode}_{color}.mp4")
    # Fichero WAV de salida
    output_wav = os.path.join(export_dir, f"recon_{base_name}_{map_mode}_{color}.wav")
    
    # 2) Instanciar decoder
    dec = PixelSoundsDecoder(
        frames_dir=frames_dir, # Donde guardar los frames extraidos
        output_wav=output_wav, # Video del que sacar los frames
        map_mode=map_mode,     # Modo
        window_type=window_type # Tipo de ventana
    )
    
    # 3) Extraer frames desde el MP4
    dec.extract_all_frames(video_file)
    
    # 4) Reconstruir el audio y guardarlo
    audio_rec, fs_rec = dec.decode()
    
    # 5) Mostrar y reproducir inline
    print(f"[Notebook] Audio reconstruido ({map_mode}_{color}):")

## 4. Modo FFT (`fft`)


### Codificación

- **Transformada de Fourier (FFT)**  
  Se aplica una FFT al bloque de audio con longitud \( N \) y se centra el espectro usando un `fftshift`.

  $$
  X = \text{fftshift}\left( \text{FFT}(\text{block}) \right)
  $$

- **Extracción de magnitud y fase**  
  $$
  \text{mag} = |X| \qquad \text{phase} = \angle X
  $$

- **Normalización a enteros sin signo de 8 bits**  
  - Magnitud:

    $$
    \text{mag}_n = \left\lfloor \frac{\text{mag}}{\max(\text{mag}) + \varepsilon} \times 255 \right\rfloor
    $$

  - Fase (rango [\(-\pi\), \(+\pi\)] → [0, 255]):

    $$
    \text{phase}_n = \left\lfloor \frac{\text{phase} + \pi}{2\pi} \times 255 \right\rfloor
    $$

- **Construcción de la imagen**  
  - **Escala de grises**:  
    Se intercalan las filas de magnitud y fase:

    $$
    \text{img}_{2k} = \text{mag}_n \quad , \quad \text{img}_{2k+1} = \text{phase}_n
    $$

  - **Modo color**:  
    Se asigna:

    $$
    R = \text{mag}_n, \quad G = \text{phase}_n, \quad B = 255
    $$

---

In [ ]:
# 3.1 Generar → Empaquetar → Decodificar para FFT (gris y color)
map_mode = "fft"
for color in ("gris", "color"):
    # 3.1.1 Carpetas de salida
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    os.makedirs(frames_dir, exist_ok=True)
    os.makedirs(export_dir, exist_ok=True)

    # 3.1.2 CODIFICAR
    enc = PixelSoundsEncoder(
        audio_path=audio_path,
        frames_dir=frames_dir,
        export_dir=export_dir,
        fps=fps,
        color_mode=color,
        map_mode=map_mode,
        window_type=window_type
    )
    enc.generate_frames()

    # 3.1.3 EMPAQUETAR VÍDEO
    video_file = enc.encode_video()
    print(f"[Notebook] Vídeo generado en: {video_file}")
    # display(Video(video_file, embed=True, width=480, height=360))

### Visualización


Los vídeos generados en modo **FFT** suelen ser muy grandes y a veces el notebook tarda o se “congela” al intentar renderizarlos inline. Por ello **no recomendamos** ejecutar la celda de abajo en el notebook si ves que no responde. 

> **Sugerencia**:  
> - Abre el fichero MP4 directamente en **VLC** o **VSCode**  
> - O puedes reproducirlos en un navegador externo


In [ ]:
base = os.path.splitext(os.path.basename(audio_path))[0]

for map_mode, color in [("fft", "gris"), ("fft", "color")]:
    video_file = os.path.join(
        "exports",
        f"{map_mode}_{color}",
        f"{base}_{map_mode}_{color}.mp4"
    )
    print(f"Vídeo FFT ({map_mode}_{color}): {video_file}")
    # DESCOMENTA la siguiente si quieres intentarlo (no lo recomendamos ) 
    # display(Video(video_file, embed=True, width=480, height=360

### Decodificación

- **Lectura de magnitud y fase**  
  - En modo color se toman los canales R y G.  
  - En modo gris se extraen de las filas 0 (magnitud) y 1 (fase).

- **Desnormalización de magnitud y fase**  
  $$
  \text{mag} = \frac{\text{mag}_n}{255}, \qquad \text{phase} = \frac{\text{phase}_n}{255} \times 2\pi - \pi
  $$

- **Reconstrucción del espectro complejo**  
  $$
  X = \text{mag} \cdot e^{j \cdot \text{phase}}
  $$

- **Transformada inversa y desfase de espectro**  
  $$
  X = \text{ifftshift}(X), \quad \text{block} = \text{Re}\left( \text{IFFT}(X) \right)
  $$

- **Solapamiento y normalización**  
  Se aplica overlap-add con ventana y se normaliza por los pesos acumulados, como en los otros modos.

El modo `fft` permite recuperar tanto el contenido espectral como la fase del bloque, generando reconstrucciones más fieles pero a costa de mayor complejidad.

In [ ]:
map_mode = "fft"
base_name = os.path.splitext(os.path.basename(audio_path))[0]

for color in ("gris", "color"):
    # 1) Directorios y rutas
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    # Nombre del vídeo que acabamos de generar
    video_file = os.path.join(export_dir, f"{base_name}_{map_mode}_{color}.mp4")
    # Fichero WAV de salida
    output_wav = os.path.join(export_dir, f"recon_{base_name}_{map_mode}_{color}.wav")
    
    # 2) Instanciar decoder
    dec = PixelSoundsDecoder(
        frames_dir=frames_dir, # Donde guardar los frames extraidos
        output_wav=output_wav, # Video del que sacar los frames
        map_mode=map_mode,     # Modo
        window_type=window_type # Tipo de ventana
    )
    
    # 3) Extraer frames desde el MP4
    dec.extract_all_frames(video_file)
    
    # 4) Reconstruir el audio y guardarlo
    audio_rec, fs_rec = dec.decode()
    
    # 5) Mostrar y reproducir inline
    print(f"[Notebook] Audio reconstruido ({map_mode}_{color}):")

## 5. Modo FIR (`fir`)


#### Codificación

- **Filtrado en tres bandas**  
  Se aplica un banco de filtros FIR al bloque:

  $$
  y_L = \text{lfilter}(b_{\text{low}}, 1, \text{block}) \\
  y_B = \text{lfilter}(b_{\text{band}}, 1, \text{block}) \\
  y_H = \text{lfilter}(b_{\text{high}}, 1, \text{block})
  $$

- **Clipping y cuantificación a enteros de 8 bits con signo**  
  Se limita cada banda al rango \([-1, +1]\) y se escala a 8 bits con signo:

  $$
  r = \left\lfloor y_L \cdot 127 \right\rfloor, \quad
  g = \left\lfloor y_B \cdot 127 \right\rfloor, \quad
  b = \left\lfloor y_H \cdot 127 \right\rfloor
  $$

  Posteriormente se reinterpretan como `uint8` para almacenarlos en la imagen:

  $$
  r_8 = \text{reinterpretar como uint8}(r), \quad \text{etc.}
  $$

- **Construcción de la imagen**  
  - **Modo color**:  
    Se construye un frame RGB directamente con los canales \((r_8, g_8, b_8)\).

  - **Modo gris**:  
    Se intercalan las tres bandas en las filas de la imagen:

    $$
    \text{img}_{3k} = r_8, \quad \text{img}_{3k+1} = g_8, \quad \text{img}_{3k+2} = b_8
    $$

In [ ]:
# 3.1 Generar → Empaquetar → Decodificar para FFT (gris y color)
map_mode = "fir"
for color in ("gris", "color"):
    # 3.1.1 Carpetas de salida
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    os.makedirs(frames_dir, exist_ok=True)
    os.makedirs(export_dir, exist_ok=True)

    # 3.1.2 CODIFICAR
    enc = PixelSoundsEncoder(
        audio_path=audio_path,
        frames_dir=frames_dir,
        export_dir=export_dir,
        fps=fps,
        color_mode=color,
        map_mode=map_mode,
        window_type=window_type
    )
    enc.generate_frames()

    # 3.1.3 EMPAQUETAR VÍDEO
    video_file = enc.encode_video()
    print(f"[Notebook] Vídeo generado en: {video_file}")
    display(Video(video_file, embed=True, width=480, height=360))

---

#### Decodificación

- **Extracción de las bandas**  
  - En **modo color**:

    $$
    y_L = \text{reinterpretar como int8}(R) / 127 \\
    y_B = \text{reinterpretar como int8}(G) / 127 \\
    y_H = \text{reinterpretar como int8}(B) / 127
    $$

  - En **modo gris**:

    $$
    y_L = \text{reinterpretar como int8}(\text{fila }0::3) / 127 \\
    y_B = \text{reinterpretar como int8}(\text{fila }1::3) / 127 \\
    y_H = \text{reinterpretar como int8}(\text{fila }2::3) / 127
    $$

- **Suma de las bandas**  
  Se combinan las tres bandas para reconstruir el bloque:

  $$
  \text{block} = y_L + y_B + y_H
  $$

- **Solapamiento y reconstrucción final**  
  Se hace overlap-add y normalización con la ventana, como en los otros modos.

  Este modo es útil para representar la energía en distintas bandas del espectro, con menor fidelidad que `fft` pero menor tamaño y buena separación frecuencial.


In [ ]:
map_mode = "fir"
base_name = os.path.splitext(os.path.basename(audio_path))[0]

for color in ("gris", "color"):
    # 1) Directorios y rutas
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    # Nombre del vídeo que acabamos de generar
    video_file = os.path.join(export_dir, f"{base_name}_{map_mode}_{color}.mp4")
    # Fichero WAV de salida
    output_wav = os.path.join(export_dir, f"recon_{base_name}_{map_mode}_{color}.wav")
    
    # 2) Instanciar decoder
    dec = PixelSoundsDecoder(
        frames_dir=frames_dir, # Donde guardar los frames extraidos
        output_wav=output_wav, # Video del que sacar los frames
        map_mode=map_mode,     # Modo
        window_type=window_type # Tipo de ventana
    )
    
    # 3) Extraer frames desde el MP4
    dec.extract_all_frames(video_file)
    
    # 4) Reconstruir el audio y guardarlo
    audio_rec, fs_rec = dec.decode()
    
    # 5) Mostrar y reproducir inline
    print(f"[Notebook] Audio reconstruido ({map_mode}_{color}):")